In [1]:
## QICK LIBRARY
from qick import *

from random import random
import time
import pickle
import numpy as np

## tProc Assembler
from qick.tprocv2_assembler import Assembler, Logger

##### Load FGPA BitStream

##################
#soc = QickSoc('./qick_216_B.bit')
soc = QickSoc('./qick_216_B1.bit')



resetting clocks: 245.76 491.52


In [2]:
print('---------------------------------------------')
print('Loading Data Memory:')
Data_qty     = 10

wr_buff = []
for i in range(Data_qty):
    a = int(random()*999999)
    wr_buff.append( [a, 0,0,0,0,0,0,0] )
    ## ERROR, value are 64 bits
    #wr_buff.append( [i, i,0,0,0,0,0,0] )
print(np.array(wr_buff))

## load_mem(mem_sel, buff_in, addr):
soc.tproc.load_mem(2, wr_buff, 0)

print('---------------------------------------------')
print("Reading Data Memory:")
#read_mem(mem_sel, addr, length):
rd_buff = soc.tproc.read_mem(2, 0, Data_qty)
print(rd_buff)

print('---------------------------------------------')

d_mem_read  = np.array(rd_buff)
d_mem_write = np.array(wr_buff)
error = np.max(d_mem_write - d_mem_read)
if error == 0:
    print('Load and Read Succesfully')
else:    
    print('Error')


---------------------------------------------
Loading Data Memory:
[[270047      0      0      0      0      0      0      0]
 [477052      0      0      0      0      0      0      0]
 [303044      0      0      0      0      0      0      0]
 [777248      0      0      0      0      0      0      0]
 [860373      0      0      0      0      0      0      0]
 [537664      0      0      0      0      0      0      0]
 [118498      0      0      0      0      0      0      0]
 [604524      0      0      0      0      0      0      0]
 [866554      0      0      0      0      0      0      0]
 [346286      0      0      0      0      0      0      0]]
---------------------------------------------
Reading Data Memory:
[[270047      0      0      0      0      0      0      0]
 [477052      0      0      0      0      0      0      0]
 [303044      0      0      0      0      0      0      0]
 [777248      0      0      0      0      0      0      0]
 [860373      0      0      0      0   

In [3]:
print('---------------------------------------------')
print('Loading WaveParam Memory:')
Data_qty     = 15

wr_buff = []
for i in range(Data_qty):
    f = int(random()*99999)
    p = int(random()*99999)
    g = int(random()*128)
    e = int(random()*64)
    l = int(random()*32)
    c = int(random()*8)
    wr_buff.append( [f, p,g,e,l,c,0,0] )
print(np.array(wr_buff))

## load_mem(mem_sel, buff_in, addr):
soc.tproc.load_mem(3, wr_buff, 0)

print('---------------------------------------------')
print("Reading WaveParam Memory:")
#read_mem(mem_sel, addr, length):
rd_buff = soc.tproc.read_mem(3, 0, Data_qty)
print(rd_buff)

print('---------------------------------------------')

d_mem_read  = np.array(rd_buff)
d_mem_write = np.array(wr_buff)
error = np.max(d_mem_write - d_mem_read)
if error == 0:
    print('Load and Read Succesfully')
else:    
    print('Error')

---------------------------------------------
Loading WaveParam Memory:
[[69883 84593    73    44     5     5     0     0]
 [74100  7125    76    46     3     4     0     0]
 [72090 86247   121    14    13     2     0     0]
 [92703 64126    35     9    17     2     0     0]
 [28332  7899    42    15    22     4     0     0]
 [ 4910 78773   125    12    31     4     0     0]
 [57336 57783    66    49    11     5     0     0]
 [95240 45020    21    44    15     4     0     0]
 [46989 91559    47    50     9     7     0     0]
 [28058 62687     2    31    11     5     0     0]
 [ 4363 62929   108    39    27     5     0     0]
 [43861 36770   117    36     9     2     0     0]
 [61598 37746   104     9    16     2     0     0]
 [88626 58336    32    42    17     3     0     0]
 [46697  7085   107    37    25     7     0     0]]
---------------------------------------------
Reading WaveParam Memory:
[[69883 84593    73    44     5     5     0     0]
 [74100  7125    76    46     3     4  

In [4]:

asm = """
//TEST program
.ALIAS repeat_cnt r0
.ALIAS pulse_start  r1
.ALIAS pulse_moving_off   r3

.CONST pulse_repeat_w     #800000
.CONST pulse_w            #2000
.CONST pulse_central_off  #1000
.CONST pulse_central_w    #5
.CONST pulse_moving_w     #5
.CONST total_repeat #1000
.CONST velocity #2

INIT:
   REG_WR pulse_start imm #1000
   REG_WR pulse_moving_off imm #0

REG_WR repeat_cnt imm total_repeat
REG_WR repeat_cnt op -op(repeat_cnt-#1) -uf

TRIG_LOOP:
   REG_WR pulse_start op -op(pulse_start+#100) 
   REG_WR r_time op -op(pulse_start)
   // ENVELOPE RISE
   TRIG p0 set
      // CENTRAL PULSE
      REG_WR r_time op -op(pulse_start+pulse_central_off)
      TRIG p1 set 
      REG_WR r_time op -op(r_time+pulse_central_w)
      TRIG p1 clr 
      // MOVING PULSE
      REG_WR pulse_moving_off op -op(pulse_moving_off+velocity)
      REG_WR r_time op -op(pulse_start+pulse_moving_off)
      TRIG p2 set
      REG_WR r_time op -op(r_time+pulse_moving_w)
      TRIG p2 clr
      // ENVELOPE FALL
      REG_WR r_time op -op(pulse_start+pulse_central_off) 
      REG_WR r_time op -op(r_time+pulse_central_off) 
      TRIG p0 clr 
      REG_WR pulse_start op -op(pulse_start+pulse_repeat_w) 
   JUMP TRIG_LOOP -wr(repeat_cnt op) -op(repeat_cnt-#1) -if(NZ) -uf

END: 
    JUMP END
"""


p_txt, p_bin             = Assembler.str_asm2bin(asm)
# p_txt is the OP_CODE of the program
print(np.array(p_txt))
# p_bin is the binary data to load in the Memory
print(np.array(p_bin))
soc.tproc.Load_PMEM(p_bin)

print ("Storing p_bin in file program_mem.pkl")
pickle.dump(p_bin, open('program_mem.pkl', 'wb')) 


print('--- LOADING MEM')
soc.tproc.Load_PMEM(p_bin)

print('--- START THE PROCESSOR')
soc.tproc.proc_start()

['000_000__000___00__0_00_00______00000000000_000000_________00000000000000000000000000000000_0000000'
 '100_011__000___11__0_00_00______00000000000_000000_________00000000000000000000001111101000_0000001'
 '100_011__000___11__0_00_00______00000000000_000000_________00000000000000000000000000000000_0000011'
 '100_011__000___11__0_00_00______00000000000_000000_________00000000000000000000001111101000_0000000'
 '100_010__000___00__1__0010______00000000000_000000_____0_0000000___000000000000000000000001_0000000'
 '100_010__000___00__0__0000______00000000000_000000_____0_0000001___000000000000000001100100_0000001'
 '100_001__000___00__0__0000______00000000000_000000_____0_0000001_0_1000000_0000000000000000_1001110'
 '110_111__011___00__0_00_00______00000000001_001000______00000000000000000000000000000000_0000000'
 '100_010__000___00__0__0000______00000000000_000000_____0_0000001___000000000000001111101000_1001110'
 '110_111__011___00__0_00_00______00000000001_001001______000000000000000000

In [5]:
##### LOAD PROGRAM FROM FILE
data_filename = "program_mem.pkl"

print('---------------------------------------------')
print("Loading Program Memory From PICKLE File : " + data_filename)
p_mem = pickle.load(open(data_filename, 'rb'))

soc.tproc.Load_PMEM(p_mem)

print('---------------------------------------------')
print("Reading Program Memory:")
rd_buff = soc.tproc.read_mem(1, 0, len(p_mem) )

print('---------------------------------------------')
d_mem_read  = np.array(rd_buff).astype(np.int32)
d_mem_write = np.array(p_mem).astype(np.int32)
error = np.max(d_mem_write - d_mem_read)
if error == 0:
    print('Load and Read Succesfully')
else:    
    print('Error')


---------------------------------------------
Loading Program Memory From PICKLE File : program_mem.pkl
---------------------------------------------
Reading Program Memory:
---------------------------------------------
Load and Read Succesfully
